In [1]:
import pandas as pd
import numpy as np
import re
pd.options.mode.chained_assignment = None  # default='warn'


No regex pattern specified. Nothing done.


In [79]:
%reset_selective -f time

In [82]:
test = pd.read_csv("test2.csv",',',index_col = 'x')
print(test)
print(type(test))

                       A                               B              C  \
x                                                                         
1   [{'dayName': 'Today'   'hours': '5:30 AM to 8:00 PM'   'open': True   
2                    NaN                             NaN            NaN   
3   [{'dayName': 'Today'   'hours': '5:00 AM to 8:00 PM'   'open': True   
4                    NaN                             NaN            NaN   
5  'dayName': 'Tomorrow'   'hours': '5:00 AM to 8:00 PM'   'open': True   
6                    NaN                          Closed            NaN   

                D  
x                  
1   'holiday': ''  
2             NaN  
3   'holiday': ''  
4             NaN  
5   'holiday': ''  
6             NaN  
<class 'pandas.core.frame.DataFrame'>


In [83]:
test2 = test['B'].copy()
print(type(test2))
test2

<class 'pandas.core.series.Series'>


x
1     'hours': '5:30 AM to 8:00 PM'
2                               NaN
3     'hours': '5:00 AM to 8:00 PM'
4                               NaN
5     'hours': '5:00 AM to 8:00 PM'
6                            Closed
Name: B, dtype: object

In [113]:
time = test2.str.extract(r'(?P<Open>\d{1,2}[:]\d{2}\s\wM)').copy()
time['Close'] = test2.str.extract(r'(?P<to>\sto\s)(?P<Close>\d{1,2}[:]\d{2}\s\wM)')['Close'].copy()
time['No_service'] = test2.str.extract(r'(?P<No_service>Closed)')['No_service'].copy()
time['Open'] = pd.to_datetime(time['Open']).dt.strftime('%H:%M')
time['Close'] = pd.to_datetime(time['Close']).dt.strftime('%H:%M')
filter = (time['No_service'] == 'Closed')
time.loc[filter,('Open', 'Close')] = time.loc[filter,('Open', 'Close')].fillna('Closed')
print(time)
#print(time2)

     Open   Close No_service
x                           
1   05:30   20:00        NaN
2     NaN     NaN        NaN
3   05:00   20:00        NaN
4     NaN     NaN        NaN
5   05:00   20:00        NaN
6  Closed  Closed     Closed


In [118]:
def get_schedule(schedule):
    print(schedule)
    time = schedule.str.extract(r'(?P<Open>\d{1,2}[:]\d{2}\s\wM)').copy()
    time['Close'] = schedule.str.extract(r'(?P<to>\sto\s)(?P<Close>\d{1,2}[:]\d{2}\s\wM)')['Close'].copy()
    time['No_service'] = schedule.str.extract(r'(?P<No_service>Closed)')['No_service'].copy()
    time['Open'] = pd.to_datetime(time['Open']).dt.strftime('%H:%M')
    time['Close'] = pd.to_datetime(time['Close']).dt.strftime('%H:%M')
    filter = (time['No_service'] == 'Closed')
    time.loc[filter,('Open', 'Close')] = time.loc[filter,('Open', 'Close')].fillna('Closed')
    return time['Open'], time['Close']    


x
1     [{'dayName': 'Today'
2                      NaN
3     [{'dayName': 'Today'
4                      NaN
5    'dayName': 'Tomorrow'
6                      NaN
Name: A, dtype: object


,A,B,C,D,A_open,A_close
x,,,,,,
1,[{'dayName': 'Today','hours': '5:30 AM to 8:00 PM','open': True,'holiday': '',NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,[{'dayName': 'Today','hours': '5:00 AM to 8:00 PM','open': True,'holiday': '',NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,'dayName': 'Tomorrow','hours': '5:00 AM to 8:00 PM','open': True,'holiday': '',NaN,NaN
6,NaN,Closed,NaN,NaN,NaN,NaN


In [119]:
test['B_open'], test['B_close'] = get_schedule(test['B'])
test

x
1     'hours': '5:30 AM to 8:00 PM'
2                               NaN
3     'hours': '5:00 AM to 8:00 PM'
4                               NaN
5     'hours': '5:00 AM to 8:00 PM'
6                            Closed
Name: B, dtype: object


,A,B,C,D,A_open,A_close,B_open,B_close
x,,,,,,,,
1,[{'dayName': 'Today','hours': '5:30 AM to 8:00 PM','open': True,'holiday': '',NaN,NaN,05:30,20:00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[{'dayName': 'Today','hours': '5:00 AM to 8:00 PM','open': True,'holiday': '',NaN,NaN,05:00,20:00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,'dayName': 'Tomorrow','hours': '5:00 AM to 8:00 PM','open': True,'holiday': '',NaN,NaN,05:00,20:00
6,NaN,Closed,NaN,NaN,NaN,NaN,Closed,Closed
